Import the required packages

In [1]:
import torch
import os
from torch import nn
import torchvision  
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
batch_size=256
epochs=30

Data import

In [4]:
transform = transforms.Compose([
    #transforms.RandomHorizontalFlip(0.5),
    #transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
# 下载FashionMNIST数据集  
fmnist_train = torchvision.datasets.FashionMNIST(
    root='./data', train=True, download=True, transform=transform)
fmnist_text = torchvision.datasets.FashionMNIST(
    root='./data', train=False, download=True, transform=transform) 
# 定义数据加载器
train_loader = torch.utils.data.DataLoader(
    fmnist_train, batch_size=batch_size, shuffle=True)
text_loader = torch.utils.data.DataLoader(
    fmnist_text, batch_size=batch_size, shuffle=False)

Define the network

In [5]:
class Netcnn(nn.Module):
    def __init__(self):
        super(Netcnn, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Dropout(0.3),
            nn.Conv2d(32, 64, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Dropout(0.3)
        )
        
        
        self.fc = nn.Sequential(
            nn.Linear(64*4*4, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.conv(x)
        x = x.view(-1, 64*4*4)  # Flatten the tensor
        x = self.fc(x)
        return x
    
class Netmlp(nn.Module):
    def __init__(self):
        super(Netmlp,self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28, 256),  
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 10)
        )
    def forward(self, x):
        return self.model(x)


In [6]:
netcnn=Netcnn().to(device)
netmlp=Netmlp().to(device)
loss = nn.CrossEntropyLoss()
optimizer1 = torch.optim.Adam(netcnn.parameters(), lr=1e-4)
optimizer2 = torch.optim.Adam(netmlp.parameters(), lr=1e-4,weight_decay=1e-4)

Define the function

In [7]:
def train(epoch,net,optimizer): 
    net.train()
    train_loss=0
    for batch, (X, y) in enumerate(train_loader):
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_loss += l.item()*X.size(0)
    train_loss = train_loss/len(train_loader.dataset)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, train_loss))
        
def validate(epoch,net,optimizer):
    net.eval()
    val_loss=0
    gt_labels=[]
    pred_labels=[]
    with torch.no_grad():
        for batch, (X, y) in enumerate(text_loader):
            X, y = X.to(device), y.to(device)   
            y_hat=net(X)
            preds=torch.argmax(y_hat,1)
            gt_labels.append(y.cpu().data.numpy())
            pred_labels.append(preds.cpu().data.numpy())
            l = loss(y_hat, y)
            val_loss+=l.item()*X.size(0)
    val_loss=val_loss/len(text_loader.dataset)
    gt_labels, pred_labels = np.concatenate(gt_labels), np.concatenate(pred_labels)
    acc = np.sum(gt_labels==pred_labels)/len(pred_labels)
    print('Epoch: {} \tValidation Loss: {:.6f}, Accuracy: {:6f}'.format(epoch, val_loss, acc))
    

Train CNN Model

In [8]:
for epoch in range(1, epochs+1):
    train(epoch, netcnn, optimizer1)
    validate(epoch, netcnn, optimizer1)
save_path = "./FahionCNN.pkl"
torch.save(netcnn, save_path)


Epoch: 1 	Training Loss: 1.005033
Epoch: 1 	Validation Loss: 0.624601, Accuracy: 0.768400
Epoch: 2 	Training Loss: 0.593426
Epoch: 2 	Validation Loss: 0.529950, Accuracy: 0.809900
Epoch: 3 	Training Loss: 0.522256
Epoch: 3 	Validation Loss: 0.480018, Accuracy: 0.827900
Epoch: 4 	Training Loss: 0.477795
Epoch: 4 	Validation Loss: 0.443286, Accuracy: 0.845800
Epoch: 5 	Training Loss: 0.445716
Epoch: 5 	Validation Loss: 0.418126, Accuracy: 0.851900
Epoch: 6 	Training Loss: 0.423557
Epoch: 6 	Validation Loss: 0.395312, Accuracy: 0.861200
Epoch: 7 	Training Loss: 0.404833
Epoch: 7 	Validation Loss: 0.383179, Accuracy: 0.864200
Epoch: 8 	Training Loss: 0.391741
Epoch: 8 	Validation Loss: 0.369297, Accuracy: 0.869400
Epoch: 9 	Training Loss: 0.376240
Epoch: 9 	Validation Loss: 0.361122, Accuracy: 0.871100
Epoch: 10 	Training Loss: 0.365791
Epoch: 10 	Validation Loss: 0.349366, Accuracy: 0.875600
Epoch: 11 	Training Loss: 0.354791
Epoch: 11 	Validation Loss: 0.340688, Accuracy: 0.879100
Epoch:

Train MLP Model

In [9]:
for epoch in range(1, epochs+1):
    train(epoch,netmlp,optimizer2)
    validate(epoch,netmlp,optimizer2)
save_path = "./FahionMLP.pkl"
torch.save(netmlp, save_path)

Epoch: 1 	Training Loss: 1.318977
Epoch: 1 	Validation Loss: 0.687434, Accuracy: 0.750900
Epoch: 2 	Training Loss: 0.674261
Epoch: 2 	Validation Loss: 0.559956, Accuracy: 0.796000
Epoch: 3 	Training Loss: 0.566433
Epoch: 3 	Validation Loss: 0.503952, Accuracy: 0.819400
Epoch: 4 	Training Loss: 0.516129
Epoch: 4 	Validation Loss: 0.476583, Accuracy: 0.825600
Epoch: 5 	Training Loss: 0.482364
Epoch: 5 	Validation Loss: 0.460137, Accuracy: 0.835100
Epoch: 6 	Training Loss: 0.457860
Epoch: 6 	Validation Loss: 0.445950, Accuracy: 0.836500
Epoch: 7 	Training Loss: 0.438615
Epoch: 7 	Validation Loss: 0.428545, Accuracy: 0.844700
Epoch: 8 	Training Loss: 0.423885
Epoch: 8 	Validation Loss: 0.419911, Accuracy: 0.847600
Epoch: 9 	Training Loss: 0.413067
Epoch: 9 	Validation Loss: 0.410816, Accuracy: 0.850200
Epoch: 10 	Training Loss: 0.399553
Epoch: 10 	Validation Loss: 0.403696, Accuracy: 0.852200
Epoch: 11 	Training Loss: 0.390782
Epoch: 11 	Validation Loss: 0.397937, Accuracy: 0.852900
Epoch: